# M3. Actividad 1
## Intersección de cuatro vias

### Equipo 2:
- Carlos Tadeo Pérez Capistrán, A01197315
- David Fernando Armendáriz Torres, A01570813
- Nicolás Herrera Hernandez, A01114972




## Imports
 

In [99]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid, MultiGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## Crear el modelo y los agentes


In [144]:
# Agente de la aspiradora o robot limpiador
class Automovil(Agent):
  def __init__(self, unique_id, model, direction):
    super().__init__(unique_id, model)
    # Direccion 0 = right, 1 = left, 2 = down, 3 = up
    # starting_points = [(4,0), (5,9), (0,4), (9, 5)]
  
    self.direction = direction
    self.color = np.random.choice([4,5])
    #if self.pos == (4, 0):
    #  self.direction = 0
    #elif self.pos == (5, 9):
    #  self.direction = 1
    #elif self.pos == (0, 4):
    #  self.direction = 2
    #else:
    #  self.direction = 3

# Función del agente para mover a la aspiradora a una celda vecina
  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=True,
      include_center=False)
    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self, new_position)
    self.moves += 1

  def step(self):
    valorSemaforo = { (4,4): (), (4,5): (), (5,4): (), (5,5): ()}
    up = (self.pos[0] + 1, self.pos[1])
    down = (self.pos[0] - 1, self.pos[1])
    left = (self.pos[0], self.pos[1] - 1)
    right = (self.pos[0], self.pos[1] + 1)
    neighbours = self.model.grid.get_neighbors(
      self.pos,
      moore=False,
      include_center=False)

      
    # Direccion 0 = right, 1 = left, 2 = down, 3 = up
    #for n in neighbours:
    #  if (n.pos == right or n.pos == left ) and isinstance(n, Automovil) and self.direction == 0:
    #    self.model.grid.move_agent(self, self.pos)
    #  else:
    #    self.model.grid.move_agent(self, right)
    #  if (n.pos == right or n.pos == left) and isinstance(n, Automovil) and self.direction == 1:
    #    self.model.grid.move_agent(self, self.pos)
    #  else:
    #    self.model.grid.move_agent(self, left)
    #  if (n.pos == up or n.pos == left ) and isinstance(n, Automovil) and self.direction == 2:
    #    self.model.grid.move_agent(self, self.pos)
    #  else:
    #    self.model.grid.move_agent(self, down)
    #  if (n.pos == up or n.pos == down) and isinstance(n, Automovil) and self.direction == 3:
    #    self.model.grid.move_agent(self, self.pos)
    #  else:
    #    self.model.grid.move_agent(self, up)
    
    if self.direction == 0:
      self.model.grid.move_agent(self, right)
    if self.direction == 1:
      self.model.grid.move_agent(self, left)
    if self.direction == 2:
      self.model.grid.move_agent(self, down)
    if self.direction == 3:
      self.model.grid.move_agent(self, up)



# Agente de la baldosa
class Semaforo(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos y el valor de suciedad, donde 1 = sucio y 0 = limpio.
    self.color = 1
    #self.next_state = None
    #self.changes = 0

  def changeColor(self):
    self.next_state = self.color
    if self.next_state == 3 and (self.changes % 3) == 0:
      self.next_state = 3
    elif self.next_state == 3 and (self.changes % 2) != 0:
      self.next_state = 3
    else:
      self.next_state = 1

  #def step(self):
  #  self.changes += 1
  #  self.changeColor()

  #def advance(self):
  #  self.color = self.next_state

class Terreno(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos y el valor de suciedad, donde 1 = sucio y 0 = limpio.
    self.color = 0


class Interseccion(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    # Se inicializan los movimientos y el valor de suciedad, donde 1 = sucio y 0 = limpio.
    self.color = 2

class Modelo(Model):
  def __init__(self, width, height, n):
    self.grid = MultiGrid(width, height, True)
    self.schedule = SimultaneousActivation(self)
    self.running = True
    self.num_agents = n



    # Ubicamos a cada las aspiradoras en la grilla de acuerdo al numero de agentes indicado
    s1 = Semaforo(1, self)
    self.schedule.add(s1)
    self.grid.place_agent(s1, (3, 3))
    s2 = Semaforo(2, self)
    self.schedule.add(s2)
    self.grid.place_agent(s2, (3, 6))
    s3 = Semaforo(3, self)
    self.schedule.add(s3)
    self.grid.place_agent(s3, (6, 3))
    s4 = Semaforo(4, self)
    self.schedule.add(s4)
    self.grid.place_agent(s4, (6, 6))

    #starting_points = [(4,0), (5,9), (0,4), (9, 5)]
    #for i in range(4):
    #  c = Automovil(i+10, self)
    #  self.grid.place_agent(c, self.random.choice(starting_points))
    #  self.schedule.add(c)

    for i in range(2):
      for j in range(2):
        b = Interseccion(30+i*4+j, self)
        self.grid.place_agent(b, (4+i, 4+j))
        self.schedule.add(b)
        
    cont = 0
    dir = 0
    for (content, x, y) in self.grid.coord_iter():
      if self.grid.is_cell_empty((x, y)):
        if x == 4 and y not in [4, 5]:
          dir = 0
          c = Automovil((x, y), self, dir)
          self.grid.place_agent(c, (x, y))
          self.schedule.add(c)
          cont += 1
        elif x == 5 and y not in [4, 5]:
          dir = 1
          c = Automovil((x, y), self, dir)
          self.grid.place_agent(c, (x, y))
          self.schedule.add(c)
          cont += 1
        elif y == 4 and x not in [4, 5]:
          dir = 2
          c = Automovil((x, y), self, dir)
          self.grid.place_agent(c, (x, y))
          self.schedule.add(c)
          cont += 1
        elif y == 5 and y not in [4, 5]:
          dir = 3
          c = Automovil((x, y), self, dir)
          self.grid.place_agent(c, (x, y))
          self.schedule.add(c)
          cont += 1

      if cont >= self.num_agents:
        break 
      
      

    for (content, x, y) in self.grid.coord_iter():
      if self.grid.is_cell_empty((x,y)):
          a = Terreno((x, y), self)
          self.grid.place_agent(a, (x, y))
          self.schedule.add(a)

    # Aquí definimos el colector de datos para obtener el grid completo y los parametros de los agentes.
    self.datacollector = DataCollector(
    model_reporters={"Grid": self.get_grid}, agent_reporters={"Color" : "color"}
    )

  def step(self):
    self.schedule.step()
    self.datacollector.collect(self)


  def get_grid(self):

      # Generamos la grid para contener los valores
      grid = np.zeros((self.grid.width, self.grid.height))

      # Asignamos una celda a cada uno de los elementos de la grilla
      for cell in self.grid.coord_iter():
          cell_content, x, y = cell
          for agent in cell_content:
            grid[x][y] = agent.color

      return grid


# Ejecución del modelo
A continuación corremos el modelo con los valores proporcionados.

In [145]:
# Definimos el tamaño del Grid, procentaje de celdas sucias y num de agentes


# Definimos el tiempo limite de ejecucion
NUMBER_CARS = 5

# Creamos el modelo con las dimensione
empty_model = Modelo(10, 10, NUMBER_CARS)

# Registramos el tiempo de inicio

# Corremos el modelo mientras que no se cumpla el tiempo limite y que todas las celdas no esten limpias
for i in range(20):
  empty_model.step()

# Guardamos el tiempo que le tomó correr al modelo.



# Datos a analizar
Obtenemos la información que almacenó el colector, este nos entregará un DataFrame de pandas que contiene toda la información. Con esta información podremos calcular lo siguiente:

- Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
- Porcentaje de celdas limpias después del termino de la simulación.
- Número de movimientos realizados por todos los agentes.

In [147]:
# Obtenemos la información recolectada del modelo
all_grid = empty_model.datacollector.get_model_vars_dataframe()


# Graficación
Graficamos la información usando `matplotlib`

In [148]:
%%capture
# Obtenemos el numero de pasos del recolector de datos
steps = len(all_grid)
cmap = matplotlib.cm.get_cmap('viridis', 7) # Puede ser cualquier otra
cmap = cmap(np.linspace(0, 1, 7))
cmap[0] = np.array([50/256, 50/256, 50/256, 1])   # Calle en gris
cmap[1] = np.array([256/256, 0/256, 0/256, 1])    # Semáforo en rojo
cmap[2] = np.array([256/256, 256/256, 0/256, 1])  # Semáforo en amarillo
cmap[3] = np.array([0/256, 256/256, 0/256, 1])    # Semáforo en verde
cmap[4] = np.array([0/256, 50/256, 100/256, 1])   # Carro azul
cmap[5] = np.array([230/256, 100/256, 20/256, 1]) # Carro naranja
cmap[6] = np.array([96/256, 170/256, 70/256, 1])  # Pasto verde

new_cmap = matplotlib.colors.ListedColormap(cmap)

fig, axs = plt.subplots(figsize=(10, 10))
axs.set_xticks([])
axs.set_yticks([])

patch = plt.imshow(all_grid.iloc[0][0], cmap=new_cmap)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])


anim = animation.FuncAnimation(fig, animate, frames=steps)

# Animación
Colores de celda:
- Gris: RobotLimpiador.
- Blanco: Baldosa limpia.
- Negro: Baldosa sucia. 

In [149]:
anim